In [1]:
import numpy as np

from qtm.lattice import RealLattice, ReciLattice
from qtm.gspace import GSpace, GkSpace

from qtm.constants import RYDBERG

# Iron Lattice with 25 Ry KE cutoff (for wavefun)
alat = 5.107
latvec_alat = 0.5 * np.array([
    [ 1,  1,  1],
    [-1,  1,  1],
    [-1, -1,  1]
])

ecutwfn = 25 * RYDBERG

reallat = RealLattice.from_alat(alat, *latvec_alat)
recilat = ReciLattice.from_reallat(reallat)

grho = GSpace(recilat, 4 * ecutwfn)
gwfn = GkSpace(grho, [0., 0., 0.])

In [2]:
from qtm.gspace.fft.utils import idxgrid2cryst

# Generating the data file for Wavefun's G-Sphere
gspc = gwfn

# Aliasing required attributes
tpiba = gspc.recilat.tpiba
nx, ny, nz = gspc.grid_shape
numg = gspc.size_g
ix, iy, iz = gspc.g_cryst

ecut = gspc.ecutwfn if isinstance(gspc, GkSpace) else gspc.ecut
rsphere = np.sqrt(2 * ecut) / tpiba

In [3]:
# Sticks are lines parallel to one of the lattice vectors of the
# reciprocal space that contain atleast one lattice point
# that is within the energy cutoff (i.e G-vectors)
# In QTM, the sticks are aligned along $b_1$, which corresponds
# to x-Axis/1st dimension in crystal repr.

# Finding (y, z) coordinates of all points to determine
# the position of sticks
iyz = nz * (iy + ny * (iy < 0)) \
    + (iz + nz * (iz < 0))
iyz_sticks = np.unique(iyz)

iy_sticks = iyz_sticks // nz
iy_sticks -= ny * (iy_sticks >= ny // 2)
iz_sticks = iyz_sticks % nz
iz_sticks -= nz * (iz_sticks >= nz // 2)

# Each point/stick is assigned 0, 1, or 2 that corresponds
# to color red, green and blue respectively
c_sticks = iyz_sticks % 3
c_points = iyz % 3
# c_sticks = np.arange(len(iyz_sticks), dtype='i8') % 3
# c_points = np.searchsorted(iyz_sticks, iyz) % 3

# The crystal coordinates and the assigned color integer
# is stored as arrays to be written to file
dat_sticks = np.array([iy_sticks, iz_sticks, c_sticks]).T
n_sticks = dat_sticks.shape[0]
dat_points = np.array([ix, iy, iz, c_points]).T
n_points = dat_points.shape[0]

In [4]:
# The G-sphere data is written to 'gspc.txt' 
# which will be read by Asymptote to generate plots

fname = 'gspc.txt'
with open(fname, 'w+') as f:
    # First the reciprocal lattice vectors (in tpiba units) are written to file
    for ax in recilat.axes_tpiba:
        f.write(f"{ax[0]:.3f} {ax[1]:.3f} {ax[2]:.3f}\n")
    # The value of tpiba is also written for reference/scaling
    f.write(f"{recilat.tpiba:.3f}\n")

    # Then the cutoff radius and the FFT grid size is written
    f.write(f"{rsphere} {nx} {ny} {nz}\n")
    
    # Writing G-vector data
    f.write(f"{n_points}\n")
    np.savetxt(f, dat_points, 
               fmt=["%.3f", "%.3f", "%.3f", "%d"])
    
    # Writing sticks data
    f.write(f"{n_sticks}\n")
    np.savetxt(f, dat_sticks,
              fmt=["%.3f", "%.3f", "%d"])

In [5]:
asy_parseinp = """
// Specifying input file
file fin=input('gspc.txt');

// Reading reciprocal lattice vectors and tpiba
triple recvec_1 = (fin, fin, fin);
triple recvec_2 = (fin, fin, fin);
triple recvec_3 = (fin, fin, fin);
real tpiba = fin;

// Constructing transform3 for
// crystal to cartesian transformations
transform3 recilat = copy(identity4);
recilat[0][0] = recvec_1.x;
recilat[1][0] = recvec_1.y;
recilat[2][0] = recvec_1.z;
recilat[0][1] = recvec_2.x;
recilat[1][1] = recvec_2.y;
recilat[2][1] = recvec_2.z;
recilat[0][2] = recvec_3.x;
recilat[1][2] = recvec_3.y;
recilat[2][2] = recvec_3.z;
transform3 recilat_inv = inverse(recilat);

// Reading cutoff radius and FFT grid shape
real rsphere = fin;
triple boxshape = (fin, fin, fin);

// Reading G-vectors data
int ng = fin;
triple[] gcryst = new triple[ng];
real[] gcol = new real[ng];
for (int i=0; i<ng; ++i) {
    gcryst[i] = (fin, fin, fin);
    gcol[i] = fin;
}

// Reading G-sticks data
int ns = fin;
pair[] scryst = new pair[ns];
real[] scol = new real[ns];
for (int i=0; i<ns; ++i) {
    scryst[i] = (fin, fin);
    scol[i] = fin;
}
//--------------------------------------------------------
// All data read from file; now we can generate the plot
"""

In [6]:
asy_config = """
import three;
settings.outformat="pdf";
settings.render=4;
size(10cm,0);
"""

In [7]:
asy_fname = 'plot_cart.asy'
asy_src = asy_parseinp + asy_config + """
// Since coordinates are in tpiba units, values like pen
// linewidths and axes lengths need to be rescaled when different
// coordinate units are used
real uscale = 1.0;


// ---------- Camera ----------
// For cartesian plot, we would like the sticks to be aligned
// vertically, and the camera to be within the first octant
real cameradist = 10 * uscale;
currentprojection = perspective(
    scale3(cameradist)
    * unit(2*recvec_2 + recvec_3 + 1*recvec_1),
    up=recvec_1
);

// ---------- FFT Meshgrid ----------
// Drawing the FFT Box (in cartesian)
draw(recilat * box(
    (-0.5 * boxshape.x, -0.5 * boxshape.y, -0.5 * boxshape.z),
    (+0.5 * boxshape.x, +0.5 * boxshape.y, +0.5 * boxshape.z)
    ), black);

// Drawing reciprocal lattice vectors as axes
real axscale = 15. * uscale;  // Length of the axis vectors
real axsize = 0.5 * uscale;  // Thickness of the axis vector lines
Label L1 = Label("$b_1$", position=EndPoint);
draw(O--scale3(axscale)*unit(recvec_1), 
    red+linewidth(axsize), 
    arrow=Arrow3(), L=L1);

Label L2 = Label("$b_2$", position=EndPoint);
draw(O--scale3(axscale)*unit(recvec_2),
    green+linewidth(axsize),
    arrow=Arrow3(), L=L2);

Label L3 = Label("$b_3$", position=EndPoint);
draw(O--scale3(axscale)*unit(recvec_3),
    blue+linewidth(axsize),
    arrow=Arrow3(), L=L3); 

// ---------- G-vectors ----------
// Plotting G-vectors as colored points
real dotsize = 2.0 * uscale;  // Size of each point

triple point;
real c;
for (int i=0; i<ng; ++i) {
    point = recilat * gcryst[i];
    c = gcol[i];
    // 0 - Red, 1 - Green, 2 - Blue, else - Black
    if (c == 0) {
        dot(point, red+linewidth(dotsize));
    } else if (c == 1) {
        dot(point, green+linewidth(dotsize));
    } else if (c == 2) {
        dot(point, blue+linewidth(dotsize));
    } else {
        dot(point, black+linewidth(dotsize));
    }
}

// ---------- Cutoff Sphere ----------
draw(scale3(rsphere) * unitsphere,
     surfacepen=white+opacity(0.3 * uscale),
     meshpen=gray(0.4 * uscale));
     
// ---------- FFT Sticks ----------
real linesize = 0.1;
triple line_start, line_end;
for (int i=0; i<ns; ++i) {
    line_start = recilat * (-0.5*boxshape.x, scryst[i].x, scryst[i].y);
    line_end   = recilat * (+0.5*boxshape.x, scryst[i].x, scryst[i].y);
    c = scol[i];
    
    // 0 - Red, 1 - Green, 2 - Blue, else - Black
    if (c == 0) {
        draw(line_start--line_end, red+linewidth(linesize));
    } else if (c == 1) {
        draw(line_start--line_end, green+linewidth(linesize));
    } else if (c == 2) {
        draw(line_start--line_end, blue+linewidth(linesize));
    } else {
        draw(line_start--line_end, black+linewidth(linesize));
    }
}
"""

with open(asy_fname, 'w') as file:
    file.write(asy_src)

In [8]:
!asy {asy_fname}

In [9]:
asy_fname = 'plot_cryst.asy'
asy_src = asy_parseinp + asy_config + """
// Overwriting recvec and recilat with unit vectors ()
// for crystal coordinates
real uscale = 1.0 / tpiba;


// ---------- Camera ----------
// For cartesian plot, we would like the sticks to be aligned
// vertically, and the camera to be within the first octant
real cameradist = 10 * uscale;
currentprojection = perspective(
    scale3(cameradist)
    * unit(2*Y + Z + 1*X),
    up=X
);

// ---------- FFT Meshgrid ----------
// Drawing the FFT Box (in cartesian)
draw(box(
    (-0.5 * boxshape.x, -0.5 * boxshape.y, -0.5 * boxshape.z),
    (+0.5 * boxshape.x, +0.5 * boxshape.y, +0.5 * boxshape.z)
    ), black);

// Drawing reciprocal lattice vectors as axes
real axscale = 15. * uscale;  // Length of the axis vectors
real axsize = 0.5 * uscale;  // Thickness of the axis vector lines
Label L1 = Label("$X$", position=EndPoint);
draw(O--scale3(axscale)*X, 
    red+linewidth(axsize), 
    arrow=Arrow3(), L=L1);

Label L2 = Label("$Y$", position=EndPoint);
draw(O--scale3(axscale)*Y,
    green+linewidth(axsize),
    arrow=Arrow3(), L=L2);

Label L3 = Label("$Z$", position=EndPoint);
draw(O--scale3(axscale)*Z,
    blue+linewidth(axsize),
    arrow=Arrow3(), L=L3); 

axscale = axscale * 0.75;
L1 = Label("$b_1$", position=EndPoint);
draw(O--scale3(axscale)*unit(recvec_1), 
    dashed+red+linewidth(axsize), 
    arrow=Arrow3(), L=L1);

L2 = Label("$b_2$", position=EndPoint);
draw(O--scale3(axscale)*unit(recvec_2),
    dashed+green+linewidth(axsize),
    arrow=Arrow3(), L=L2);

L3 = Label("$b_3$", position=EndPoint);
draw(O--scale3(axscale)*unit(recvec_3),
    dashed+blue+linewidth(axsize),
    arrow=Arrow3(), L=L3); 

// ---------- G-vectors ----------
// Plotting G-vectors as colored points
real dotsize = 2.0 * uscale;  // Size of each point

triple point;
real c;
for (int i=0; i<ng; ++i) {
    point = gcryst[i];
    c = gcol[i];
    // 0 - Red, 1 - Green, 2 - Blue, else - Black
    if (c == 0) {
        dot(point, red+linewidth(dotsize));
    } else if (c == 1) {
        dot(point, green+linewidth(dotsize));
    } else if (c == 2) {
        dot(point, blue+linewidth(dotsize));
    } else {
        dot(point, black+linewidth(dotsize));
    }
}

// ---------- Cutoff Sphere ----------
draw(recilat_inv * scale3(rsphere) * unitsphere,
     surfacepen=white+opacity(0.3 * uscale),
     meshpen=gray(0.4 * uscale));
     
// ---------- FFT Sticks ----------
real linesize = 0.1;
triple line_start, line_end;
for (int i=0; i<ns; ++i) {
    line_start = (-0.5*boxshape.x, scryst[i].x, scryst[i].y);
    line_end   = (+0.5*boxshape.x, scryst[i].x, scryst[i].y);
    c = scol[i];
    
    // 0 - Red, 1 - Green, 2 - Blue, else - Black
    if (c == 0) {
        draw(line_start--line_end, red+linewidth(linesize));
    } else if (c == 1) {
        draw(line_start--line_end, green+linewidth(linesize));
    } else if (c == 2) {
        draw(line_start--line_end, blue+linewidth(linesize));
    } else {
        draw(line_start--line_end, black+linewidth(linesize));
    }
}
"""

with open(asy_fname, 'w') as file:
    file.write(asy_src)

In [10]:
!asy {asy_fname}

In [ ]:
asy_fname = 'plot_igrid.asy'
asy_src = asy_parseinp + asy_config + """
// Overwriting recvec and recilat with unit vectors ()
// for crystal coordinates
real uscale = 1.0 / tpiba;


// ---------- Camera ----------
// For cartesian plot, we would like the sticks to be aligned
// vertically, and the camera to be within the first octant
real cameradist = 10 * uscale;
currentprojection = perspective(
    scale3(cameradist)
    * unit(2*Y + Z + 1*X),
    up=X
);

// ---------- FFT Meshgrid ----------
// Drawing the FFT Box (in cartesian)
draw(box(
    (-0.5 * boxshape.x, -0.5 * boxshape.y, -0.5 * boxshape.z),
    (+0.5 * boxshape.x, +0.5 * boxshape.y, +0.5 * boxshape.z)
    ), black);

// Drawing reciprocal lattice vectors as axes
real axscale = 15. * uscale;  // Length of the axis vectors
real axsize = 0.5 * uscale;  // Thickness of the axis vector lines
Label L1 = Label("$X$", position=EndPoint);
draw(O--scale3(axscale)*X, 
    red+linewidth(axsize), 
    arrow=Arrow3(), L=L1);

Label L2 = Label("$Y$", position=EndPoint);
draw(O--scale3(axscale)*Y,
    green+linewidth(axsize),
    arrow=Arrow3(), L=L2);

Label L3 = Label("$Z$", position=EndPoint);
draw(O--scale3(axscale)*Z,
    blue+linewidth(axsize),
    arrow=Arrow3(), L=L3); 

axscale = axscale * 0.75;
L1 = Label("$b_1$", position=EndPoint);
draw(O--scale3(axscale)*unit(recvec_1), 
    dashed+red+linewidth(axsize), 
    arrow=Arrow3(), L=L1);

L2 = Label("$b_2$", position=EndPoint);
draw(O--scale3(axscale)*unit(recvec_2),
    dashed+green+linewidth(axsize),
    arrow=Arrow3(), L=L2);

L3 = Label("$b_3$", position=EndPoint);
draw(O--scale3(axscale)*unit(recvec_3),
    dashed+blue+linewidth(axsize),
    arrow=Arrow3(), L=L3); 

// ---------- G-vectors ----------
// Plotting G-vectors as colored points
real dotsize = 2.0 * uscale;  // Size of each point

triple point;
real c;
for (int i=0; i<ng; ++i) {
    point = gcryst[i];
    if (point.x > boxshape.x / 2) {
        point.x = point.x - boxshape.x
    }
    if (point.y > boxshape.y / 2) {
        point.y = point.y - boxshape.y
    }
    if (point.z > boxshape.z / 2) {
        point.z = point.z - boxshape.z
    }
    c = gcol[i];
    // 0 - Red, 1 - Green, 2 - Blue, else - Black
    if (c == 0) {
        dot(point, red+linewidth(dotsize));
    } else if (c == 1) {
        dot(point, green+linewidth(dotsize));
    } else if (c == 2) {
        dot(point, blue+linewidth(dotsize));
    } else {
        dot(point, black+linewidth(dotsize));
    }
}

// ---------- Cutoff Sphere ----------
draw(recilat_inv * scale3(rsphere) * unitsphere,
     surfacepen=white+opacity(0.3 * uscale),
     meshpen=gray(0.4 * uscale));
     
"""

with open(asy_fname, 'w') as file:
    file.write(asy_src)